## Convolutional graphs
###### Using np to realize backward propagation in convolutional network

In [1]:
import numpy as np

##### Testing summing up operations

In [2]:
np.random.seed(0)

In [3]:
H = 4
W = 3

In [4]:
x = np.random.randn(H,W)
y = np.random.randn(H,W)
z = np.random.randn(H,W)
print(x,'\n',y,'\n',z)

[[ 1.76405235  0.40015721  0.97873798]
 [ 2.2408932   1.86755799 -0.97727788]
 [ 0.95008842 -0.15135721 -0.10321885]
 [ 0.4105985   0.14404357  1.45427351]] 
 [[ 0.76103773  0.12167502  0.44386323]
 [ 0.33367433  1.49407907 -0.20515826]
 [ 0.3130677  -0.85409574 -2.55298982]
 [ 0.6536186   0.8644362  -0.74216502]] 
 [[ 2.26975462 -1.45436567  0.04575852]
 [-0.18718385  1.53277921  1.46935877]
 [ 0.15494743  0.37816252 -0.88778575]
 [-1.98079647 -0.34791215  0.15634897]]


In [5]:
a = x * y
b = a + z
print(a,'\n',b)

[[ 1.34251038  0.04868913  0.43442581]
 [ 0.74772853  2.79027931  0.20049663]
 [ 0.297442    0.12927355  0.26351668]
 [ 0.26837482  0.12451648 -1.07931093]] 
 [[ 3.61226501 -1.40567654  0.48018432]
 [ 0.56054468  4.32305853  1.6698554 ]
 [ 0.45238942  0.50743607 -0.62426907]
 [-1.71242165 -0.22339567 -0.92296196]]


In [6]:
c = np.sum(b)
print(c)

6.7170085378


###### Using back path to compute derivatives

In [7]:
grad_c = 1.
grad_b = grad_c * np.ones((H,W))
grad_a = grad_b.copy()
print(grad_c)
print(grad_b)
print(grad_a)

1.0
[[ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]]
[[ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]]


In [8]:
grad_z = grad_b.copy()
grad_x = grad_a*y
grad_y = grad_a*x
print(grad_z)
print(grad_y)
print(grad_x)

[[ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]
 [ 1.  1.  1.]]
[[ 1.76405235  0.40015721  0.97873798]
 [ 2.2408932   1.86755799 -0.97727788]
 [ 0.95008842 -0.15135721 -0.10321885]
 [ 0.4105985   0.14404357  1.45427351]]
[[ 0.76103773  0.12167502  0.44386323]
 [ 0.33367433  1.49407907 -0.20515826]
 [ 0.3130677  -0.85409574 -2.55298982]
 [ 0.6536186   0.8644362  -0.74216502]]


###### The disadvantage of numpy is that (1) it cannot be used in GPU; (2) have to compute our own gradients one by one

###### Basic convolutional graph using tensor flow:

In [9]:
import numpy as np
np.random.seed(0)
import tensorflow as tf

In [10]:
H, W = 3, 4
print (H,W)
print (type(H), type(W))

3 4
<class 'int'> <class 'int'>


In [11]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = tf.placeholder(tf.float32)
print(x,y,z)

Tensor("Placeholder:0", dtype=float32) Tensor("Placeholder_1:0", dtype=float32) Tensor("Placeholder_2:0", dtype=float32)


In [12]:
a = x * y
b = a + z
print(a,b)

Tensor("mul:0", dtype=float32) Tensor("add:0", dtype=float32)


In [13]:
c = tf.reduce_sum(b)
print(c)

Tensor("Sum:0", dtype=float32)


In [14]:
grad_x, grad_y, grad_Z = tf.gradients(c, [x,y,z])
print(grad_x,grad_y,grad_Z)

Tensor("gradients/mul_grad/Reshape:0", dtype=float32) Tensor("gradients/mul_grad/Reshape_1:0", dtype=float32) Tensor("gradients/add_grad/Reshape_1:0", dtype=float32)


In [23]:
with tf.Session() as sess:
    values = {
        x : np.random.randn(H,W),
        y : np.random.randn(H,W),
        z : np.random.randn(H,W),
    }
    out = sess.run([c, grad_x, grad_y, grad_z], feed_dict = values)
    c_val, grad_x_val, grad_y_val, grad_z_val = out
    print (c_val)

TypeError: Fetch argument Variable containing:
 5.2881
[torch.FloatTensor of size 1]
 has invalid type <class 'torch.autograd.variable.Variable'>, must be a string or Tensor. (Can not convert a Variable into a Tensor or Operation.)

###### Trying pytorch

In [15]:
import torch

In [16]:
from torch.autograd import Variable

In [17]:
W, H = 3, 4
x = Variable(torch.randn(W, H), requires_grad = True)
y = Variable(torch.randn(W, H), requires_grad = True)
z = Variable(torch.randn(W, H), requires_grad = True)
print(x,y,z)

Variable containing:
 0.4226  2.0878 -0.0147 -0.0382
 0.7552  0.9396 -0.6250  0.3414
-0.2681 -0.1489 -0.2944 -1.0345
[torch.FloatTensor of size 3x4]
 Variable containing:
 0.9717 -0.0945  0.8638 -0.6235
 0.4705  1.0961 -0.1530  0.1934
-1.3276  0.6956 -0.1475 -1.0059
[torch.FloatTensor of size 3x4]
 Variable containing:
-0.9368 -0.1018 -0.1213  0.5981
 0.1564  0.6644  1.5019  0.2150
 0.7569 -1.3332  0.0525  0.7280
[torch.FloatTensor of size 3x4]



In [18]:
a = x*y
b = a+z
c = torch.sum(b)
print(a,b,c)

Variable containing:
 0.4106 -0.1973 -0.0127  0.0238
 0.3553  1.0299  0.0956  0.0660
 0.3560 -0.1036  0.0434  1.0407
[torch.FloatTensor of size 3x4]
 Variable containing:
-0.5262 -0.2990 -0.1340  0.6220
 0.5118  1.6943  1.5975  0.2810
 1.1128 -1.4368  0.0960  1.7687
[torch.FloatTensor of size 3x4]
 Variable containing:
 5.2881
[torch.FloatTensor of size 1]



In [19]:
c.backward()
print(x.grad.data)
print(y.grad.data)
print(z.grad.data)


 0.9717 -0.0945  0.8638 -0.6235
 0.4705  1.0961 -0.1530  0.1934
-1.3276  0.6956 -0.1475 -1.0059
[torch.FloatTensor of size 3x4]


 0.4226  2.0878 -0.0147 -0.0382
 0.7552  0.9396 -0.6250  0.3414
-0.2681 -0.1489 -0.2944 -1.0345
[torch.FloatTensor of size 3x4]


 1  1  1  1
 1  1  1  1
 1  1  1  1
[torch.FloatTensor of size 3x4]

